In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *


#clasificadores
from utils.clasificacion import *

#otros
import os
import numpy as np
import pandas as pd
import timeit

#variables para guardar los resultados
tipo = pruebas[0]
dataset_name = datasets[0]
dims = dimensions[dataset_name]

In [3]:
print tipo
print dataset_name
print dims
print data_path

baseline
amazon
3000
data


## Pruebas con el dataset Amazon

#### El formato de cada prueba es:

* nombre_prueba (parámetro=valor)
 
#### Por ejemplo, para la prueba de mSDA con 5 capas, el nombre es:

* mSDA (l=5)

#### Las pruebas para cada método son:

* SDA: Cantidad de capas entre {1, 3, 5} con la mitad de la dimensionalidad original por cada capa y cantidad de epochs entre {25, 50}.
* mSDA: Cantidad de capas entre {1, 3, 5}.
* PCA: Ejecuciones cambiando la dimensionalidad a 1/2 y 1/4 de la original.
* GFK: Pruebas con cantidad de sub-dimensiones entre {10, 50, 100}


In [4]:
# cargando dataset Amazon
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
unlabeled = dataset_object.unlabeled
domains = dataset_object.domains

Dataset already splitted


In [5]:
X = dataset_object.get_all_X()
X.shape

(27677, 3000)

## GFK

In [ ]:
from utils.adaptacion import adapt_gfk

In [ ]:
new_times_path = os.path.join(times_path, "me2_gfk.csv")

if os.path.exists(new_times_path):
    df_gfk = pd.read_csv(new_times_path)
    i = df_gfk.shape[0]
else:
    df_gfk = pd.DataFrame(columns=dataframe_time_columns)
    i = 0

In [ ]:
size_dims = int(dims/2)
n_subs = [10, 50, 100]

tgt = domains[0]
src = domains[1]

X_tr = np.asarray(labeled[tgt]['X_tr'].todense())
y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)

X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

In [ ]:
runs = 5

for j in xrange(runs):
    print j+1
    for n in n_subs:
        t_inicio = timeit.default_timer()

        nombre = "GFK (n_dims=%d)" % n
        print nombre

        adapt_gfk(X_tr, X_ts, n, size_dims)
        
        t_fin = timeit.default_timer()
        tiempo = (t_fin-t_inicio)/60.

        print "\tEntrenado en %.3fm" % tiempo

        df_gfk.loc[i] = [nombre, tiempo]
        i = i+1

In [ ]:
df_gfk = df_gfk.groupby('Adaptacion').mean().reset_index()
df_gfk

In [ ]:
print "Guardando en %s" % new_times_path
df_gfk.to_csv(new_times_path)
print "Resultados guardados."

# mSDA Theano

In [ ]:
from mSDA.msda_theano import mSDATheano
import theano.tensor as T

In [ ]:
new_times_path = os.path.join(times_path, "me2_msda.csv")

if os.path.exists(new_times_path):
    df_msda = pd.read_csv(new_times_path)
    i = df_msda.shape[0]
else:
    df_msda = pd.DataFrame(columns=dataframe_time_columns)
    i = 0

In [ ]:
layers = [1, 3, 5]
x = T.dmatrix('x')
runs = 5

In [ ]:
for j in xrange(runs):
    print j+1
    for layer in layers:
        nombre = "mSDA (l=%d)" % layer
        print nombre

        msda = mSDATheano(x, layer, 0.3)
        tiempo = msda.fit(X)

        df_msda.loc[i] = [nombre, tiempo]

        i = i+1

In [ ]:
df_msda = df_msda.groupby('Adaptacion').mean().reset_index()
df_msda

In [ ]:
print "Guardando en %s" % new_times_path
df_msda.to_csv(new_times_path)
print "Resultados guardados."

# SDA

In [ ]:
from utils.adaptacion import create_SDA
from sklearn.model_selection import train_test_split

In [ ]:
new_times_path = os.path.join(times_path, "me2_sda.csv")

if os.path.exists(new_times_path):
    df_sda = pd.read_csv(new_times_path)
    i = df_sda.shape[0]
else:
    df_sda = pd.DataFrame(columns=dataframe_time_columns)
    i = 0

In [ ]:
# se divide el dataset para los datos de entrenamiento y validacion del SDA
X_train, X_val, _, _ = train_test_split(X, np.zeros(X.shape[0]), test_size=0.2, random_state=42)

print X_train.shape
print X_val.shape

In [ ]:
layers = [[dims/2],
          [dims/2, dims/2, dims/2],
          [dims/2, dims/2, dims/2, dims/2, dims/2]]

epochs = [25, 50]
runs = 5

In [ ]:
for j in xrange(runs):
    print j+1
    for epoch in epochs:
        for layer in layers:
            nombre = "SDA (l=%d, epochs=%d)" % (len(layer), epoch)
            print nombre

            autoencoder, encoder = create_SDA(X_train.shape[1], layer, 0.3)

            t_inicio = timeit.default_timer()

            print "\tEntrenando autoencoder..."
            autoencoder.fit(X_train, X_train,
               epochs=epoch,
               batch_size=256,
               shuffle=True,
               verbose=0,
               validation_data=(X_val, X_val))

            t_fin = timeit.default_timer()

            tiempo = (t_fin - t_inicio)/60.
            print "\tEntrenado en %.3fm" % tiempo

            df_sda.loc[i] = [nombre, tiempo]

            i = i+1

In [ ]:
df_sda = df_sda.groupby('Adaptacion').mean().reset_index()
df_sda

In [ ]:
print "Guardando en %s" % new_times_path
df_sda.to_csv(new_times_path)
print "Resultados guardados."

# PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
new_times_path = os.path.join(times_path, "me2_pca.csv")

if os.path.exists(new_times_path):
    df_pca = pd.read_csv(new_times_path)
    i = df_pca.shape[0]
else:
    df_pca = pd.DataFrame(columns=dataframe_time_columns)
    i = 0

In [ ]:
n_components = [dims/2, dims/4]
runs = 5

In [ ]:
for j in xrange(5):
    print j+1
    for n in n_components:
        nombre = "PCA (n=%d)" % n
        print "\t%s" % nombre

        new_model = PCA(n_components=n)
        
        t_inicio = timeit.default_timer()
        new_model.fit(X)
        t_fin = timeit.default_timer()

        tiempo = (t_fin - t_inicio)/60.
        print "\tEntrenado en %.3fm" % tiempo
        
        df_pca.loc[i] = [nombre, tiempo]
        i = i+1

In [ ]:
df_pca = df_pca.groupby('Adaptacion').mean().reset_index()
df_pca

In [ ]:
print "Guardando en %s" % new_times_path
df_pca.to_csv(new_times_path)
print "Resultados guardados."